#

# Training 

In [1]:
# train_mlp.py

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
input_size = 784  # 28x28 images
hidden_size = 128
hidden_size2 = 64
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate = 0.001

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = x.view(-1, input_size)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model, loss, and optimizer
model = MLP(input_size, hidden_size, hidden_size2, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'mlp_model.pth')
print("Model trained and saved as 'mlp_model.pth'")


Using device: cpu
Epoch [1/5], Step [100/938], Loss: 0.5518
Epoch [1/5], Step [200/938], Loss: 0.3458
Epoch [1/5], Step [300/938], Loss: 0.2943
Epoch [1/5], Step [400/938], Loss: 0.4696
Epoch [1/5], Step [500/938], Loss: 0.3476
Epoch [1/5], Step [600/938], Loss: 0.3333
Epoch [1/5], Step [700/938], Loss: 0.2942
Epoch [1/5], Step [800/938], Loss: 0.3804
Epoch [1/5], Step [900/938], Loss: 0.3346
Epoch [2/5], Step [100/938], Loss: 0.1299
Epoch [2/5], Step [200/938], Loss: 0.3247
Epoch [2/5], Step [300/938], Loss: 0.0829
Epoch [2/5], Step [400/938], Loss: 0.1879
Epoch [2/5], Step [500/938], Loss: 0.1665
Epoch [2/5], Step [600/938], Loss: 0.1751
Epoch [2/5], Step [700/938], Loss: 0.1513
Epoch [2/5], Step [800/938], Loss: 0.0423
Epoch [2/5], Step [900/938], Loss: 0.1973
Epoch [3/5], Step [100/938], Loss: 0.1802
Epoch [3/5], Step [200/938], Loss: 0.1362
Epoch [3/5], Step [300/938], Loss: 0.1135
Epoch [3/5], Step [400/938], Loss: 0.1083
Epoch [3/5], Step [500/938], Loss: 0.0911
Epoch [3/5], Ste

In [2]:
# test_mlp.py

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
input_size = 784  # 28x28 images
hidden_size = 128
hidden_size2 = 64
num_classes = 10
batch_size = 64

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, hidden_size2, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = x.view(-1, input_size)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the trained model
model = MLP(input_size, hidden_size, hidden_size2, num_classes).to(device)
model.load_state_dict(torch.load('mlp_model.pth'))
model.eval()  # Set model to evaluation mode

# Testing loop
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')


Using device: cpu
Accuracy of the model on the test images: 96.51%


In [1]:
# train_kan.py

import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import kan  # Assuming you have PyKAN installed

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "gpu")
print(f"Using device: {device}")

# Hyperparameters
input_size = 784  # 28x28 images flattened
hidden_size = 128  # Equivalent hidden size used for comparison
hidden_size2 = 64
num_classes = 10
num_epochs = 5
batch_size = 64
learning_rate = 0.001

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Define the PyKAN model
# In PyKAN, width specifies the layer structure
# We'll map input_size to hidden layers and output size similar to MLP
model = kan.KAN(width=[input_size, hidden_size, hidden_size2, num_classes], grid=10, k=3, seed=0).to(device)

# Initialize the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        images = images.view(-1, input_size)  # Flatten the images for input

        # Forward pass
        outputs = model(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Save the model
torch.save(model.state_dict(), 'kan_model.pth')
print("Model trained and saved as 'kan_model.pth'")


Using device: cuda
checkpoint directory created: ./model
saving model version 0.0


KeyboardInterrupt: 